In [3]:
import sys
from pathlib import Path

# add repo root
root = Path().resolve().parents[0]
sys.path.append(str(root))

In [10]:
from src.data.timeseries_dataset import TimeSeriesDataset
from src.config import SENTINEL_DIR, MASK_DIR

# find all *_RGBY_Mosaic.tif files
all_sentinel_files = list(SENTINEL_DIR.glob("*_RGBNIRRSWIRQ_Mosaic.tif"))

# extract ID (filename without suffix)
ids = [f.stem.replace("_RGBNIRRSWIRQ_Mosaic", "") for f in all_sentinel_files]

ds = TimeSeriesDataset(ids, sensor="sentinel", slice_mode="first_half")

x, y = ds[0]
print(x.shape)  # e.g. (7, 9, H, W) because half of 14 is 7

torch.Size([7, 9, 67, 108])


In [19]:
x, y = ds[0]
print(y.shape)  # mask dimension (H, W) 

torch.Size([67, 108])


In [26]:
x = x.float() / 10000.0

In [ ]:
#sjekker hva slags shapes bildene har
for i, image in enumerate(ds):
    x, y = image

    print(f"image nr {i}: image shape - {x.shape}, mask shape - {y.shape}")

In [27]:
from src.data.transform import RandomCropTS

# crops all images to H,W 64,64
ds.transform = RandomCropTS(64)


In [28]:
from src.models.external.torchrs_fc_cd import FCSiamDiff

model =  FCSiamDiff(
    channels=x.shape[1],  # 9 for sentinel, 3 for planet, 3 for vhr
    t=x.shape[0],         # 7 here
    num_classes=2
)
model

FCSiamDiff(
  (encoder): SiamEncoder(
    (0): ConvBlock(
      (model): Sequential(
        (0): Conv2d(9, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU()
        (3): Dropout(p=0.2, inplace=False)
        (4): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (5): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (6): ReLU()
        (7): Dropout(p=0.2, inplace=False)
      )
      (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    )
    (1): ConvBlock(
      (model): Sequential(
        (0): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU()
        (3): Dropout(p=0.2, inplace=False)
        (4): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1)

In [ ]:
from sklearn.model_selection import train_test_split

all_sentinel_files = list(SENTINEL_DIR.glob("*_RGBNIRRSWIRQ_Mosaic.tif"))
train_ids, test_ids = train_test_split(all_sentinel_files, test_size=0.1, random_state=0)
train_ids, val_ids = train_test_split(train_ids, test_size=0.1, random_state=0)
